In [14]:
# import modules
import io, os, sys, types # needed
import glob # needed
from nipype.pipeline.engine import Workflow, Node, MapNode # needed
from nipype.interfaces.utility import Function, IdentityInterface
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe
from nipype.interfaces.freesurfer import MRIConvert
from nipype.interfaces.freesurfer import ReconAll
from nipype import config

In [15]:
# specify variables

leadsdir = '/cluster/animal/scan_data/leads/'
dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
unpacklog = "/autofs/cluster/animal/scan_data/leads/recon/unpack.log"
recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
folders = [x for x in os.listdir(dicomdir) if not x.startswith(".")]
subject = 'LDS0370007_20180801'

# wipe clean batch.recon.list
open(recondir+'batch.recon.list', 'w').close()

# subjects with more than one MPRAGE:: these are the better QC ones:
specialcases = {'LDS0370006':'/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370006/Accelerated_Sagittal_MPRAGE/2018-07-26_10_41_42.0/S708999/',
                'LDS0370007':'/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370007/Accelerated_Sagittal_MPRAGE/2018-08-01_10_49_41.0/S711584/',
                'LDS0370013': '/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370013/Accelerated_Sagittal_MPRAGE/2018-08-22_08_55_38.0/S722833/'}

# make a list of one dicom per unique series
alldicomdirs = glob.glob(dicomdir+'/*/Accelerated_Sagittal_MPRAGE/*/*/')
sh_dicomlist = []
for el in alldicomdirs:
    splitstr = el.split('/')
    if "_" not in splitstr[6]:
        file =  os.listdir(el)[0] # pick any dicom
        sh_dicomlist.append(splitstr[6]+'/'+splitstr[7]+'/'+splitstr[8]+'/'+splitstr[9]+'/'+file)

# define workflow
leads_workflow = Workflow(name='leads_workflow')

# configure to stop on first crash
cfg = dict(execution={'stop_on_first_crash': True})
config.update_config(cfg)


In [3]:
# for testing
sh_dicomlist = [sh_dicomlist[0]]

In [4]:
# NODE: CREATEDIR

def createdir(val): # combined with find_dicom
    import os
    recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
    subject = val.split('/')[6]
    reconpath = recondir+subject
    MPRAGE_path = val.rsplit('/',1)[0]+'/'
    record = True
    while record == True:
        try:
            imgpath = recondir+subject+'/mri/orig/'
            os.makedirs(imgpath)
            dumplocation = imgpath+'001.mgz'
            dicomlist = os.listdir(MPRAGE_path) #can be any number; choose 0 (no longer need special cases?)
            pickdicom = MPRAGE_path+dicomlist[0]
            record = False
        except(FileExistsError):
            #pass
            subject = subject+"-"
            reconpath = recondir+subject
    #return subject, imgpath, reconpath, dicomlist, MPRAGE_path
    return reconpath, MPRAGE_path, pickdicom, dumplocation, recondir


CREATEDIR = pe.Node(Function(input_names=["val"],
                         output_names=["createdir_out1","createdir_out2", "createdir_out3", "createdir_out4", "createdir_out5"], # actual dicom (redundant to create unpacking node visualization)
                         function=createdir),
                        name='CREATEDIR')


In [5]:
# NODE : UNPACK

def unpack(subjectdir, MPRAGE_path): # combine raw_dump and parse_scaninfo?
    from os import system
    import csv
    import pandas as pd
    cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (MPRAGE_path, subjectdir, subjectdir)
    system(cmdstring)
    with open(subjectdir+'/scan.info', 'r') as in_file:
        for line in in_file:
            editline = line.split()
            with open(subjectdir+'/scaninfo.csv', 'w') as result:
                wr = csv.writer(result, dialect='excel')
                wr.writerow(editline)
            result.close()
        in_file.close()
    scan_info = pd.read_csv(subjectdir+'/scaninfo.csv', header=None)
    subname = subjectdir.split('/')[-1]
    return subname, subjectdir, scan_info

UNPACK = pe.Node(Function(input_names=["subjectdir","MPRAGE_path"],
                         output_names=["unpack_out1","unpack_out2", "unpack_out3"], # actual dicom (redundant to create unpacking node visualization)
                         function=unpack),
                        name='UNPACK')


In [6]:
# NODE : CONVERT2MGZ

CONVERT2MGZ = pe.Node(MRIConvert(out_type='mgz'),
                        name='CONVERT2MGZ')

In [7]:
# NODE RENAME_AND_LOG
# note: decided to add this afterward precaution to increase efficiency because there are few errors
# and want to run the unpack and convert2mgz in parallel)

def rename_and_log(subjectdir, scan_info, mgz, reconfolder, subname):
    import re
    import os
    import pandas as pd
    # load in the scaninfo file
    dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
    check = scan_info.iloc[0,2] # first row (only row); second col (validity)
    if check != 'ok':
        with open(reconfolder+'/scanerrors', "a") as efile:
            efile.write(scan_info.iloc[0,7]) # log for errors in dicoms (or any ommitted scans)
            # delete entire recon folder
    else:
        # find date:
        datestring = scan_info.iloc[0,7] 
        date = re.search('raw_'+r'+[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', datestring).group()[4:]
        subjectdir = reconfolder+subname+'_'+date
        #rename folders to have session date in recon and dicom dirs
        os.rename(reconfolder+subname, reconfolder+subname+'_'+date)
        os.rename(dicomdir+subname, dicomdir+subname+'_'+date)
        with open(reconfolder+'/batch.recon.list', "a") as bfile:
            bfile.write(subname+'_'+date)
        with open(reconfolder+'/unpack.log', "a") as ufile:
            ufile.write(subname+'_'+date)
        # should I makea scannotes? (will add info after recon)
        df = pd.DataFrame(Elements, columns= ['scan_notes', 'loni_overallpass', 'download_date','xnat_upload','recon_path','recon_notes','dickerson_overallpass'])
        df.to_csv(subjectdir+'/scannotes.csv')
    return subjectdir

RENAME_AND_LOG = pe.Node(Function(input_names=["subjectdir","scan_info",'mgz', 'reconfolder', 'subname'],
                         output_names=["subjectdir"],
                         function=rename_and_log),
                        name='RENAME_AND_LOG')

In [ ]:
# # NODE : RECON-ALL (https://nipype.readthedocs.io/en/0.12.1/interfaces/generated/nipype.interfaces.freesurfer.preprocess.html)

# can make a paramiko function...

# reconall = ReconAll()
# reconall.inputs.subject_id = 'foo'  # put subject id with data (output from rename_and_log)
# reconall.inputs.directive = 'all'   # keep for first recons
# reconall.inputs.subjects_dir = '.'  # put the recondir
# reconall.inputs.T1_files = 'structural.nii'  # but the .mgz file?
# reconall.cmdline
# 'recon-all -all -i structural.nii -subjid foo -sd .'

In [8]:
# # # NODE : INFOSOURCE
INFOSOURCE = Node(IdentityInterface(fields=['subject_name'], mandatory_inputs=False),
                  name="INFOSOURCE")

INFOSOURCE.iterables = ('subject_name', sh_dicomlist)

# NODE : SELECTFILES
#templates = dict(dicom=sh_dicomlist[0])    ## THIS WORKED!
templates = {
    "dicom": "{subject_name}" 
    }
SELECTFILES = Node(nio.SelectFiles(templates, base_directory=dicomdir),
                   name="SELECTFILES")

# NODE : DATASINK
DATASINK = Node(nio.DataSink(base_directory=leadsdir,
                container='recon_nip'),
                name="DATASINK")

In [9]:
# Connect all nodes (including INFOSOURCE, SELECTFILES, and DATASINK) to workflow

leads_workflow.connect([(INFOSOURCE, SELECTFILES, [('subject_name', 'subject_name')]),
                (SELECTFILES, CREATEDIR, [('dicom', 'val')]),
                (CREATEDIR, UNPACK, [('createdir_out1', 'subjectdir')]),
                 (CREATEDIR, UNPACK, [('createdir_out2', 'MPRAGE_path')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out3', 'in_file')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out4', 'out_file')]),   
                (CONVERT2MGZ, RENAME_AND_LOG, [('out_file', 'mgz')]),
                (CREATEDIR, RENAME_AND_LOG, [('createdir_out5', 'reconfolder')]),
                (UNPACK, RENAME_AND_LOG, [('unpack_out1', 'subname')]),
                (UNPACK, RENAME_AND_LOG, [('unpack_out2', 'subjectdir')]),
                (UNPACK, RENAME_AND_LOG, [('unpack_out3', 'scan_info')]),
                (RENAME_AND_LOG, DATASINK, [('subjectdir','backup')])  # backup folder?
                 ])

In [10]:
# Execute your workflow in sequential way
leads_workflow.write_graph(graph2use='flat')
leads_workflow.run(run(plugin='MultiProc', plugin_args={'n_procs' : 2})
#leads_workflow.run()

190222-14:15:25,463 nipype.workflow INFO:
	 Workflow leads_workflow settings: ['check', 'execution', 'logging', 'monitoring']
190222-14:15:25,497 nipype.workflow INFO:
	 Running serially.
190222-14:15:25,499 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.SELECTFILES" in "/tmp/tmpn2xjtgrt/leads_workflow/_subject_name_LDS0370007..Accelerated_Sagittal_MPRAGE..2018-08-01_10_49_41.0..S711583..LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180801153311710_44_S711583_I1029697.dcm/SELECTFILES".
190222-14:15:25,503 nipype.workflow INFO:
	 [Node] Running "SELECTFILES" ("nipype.interfaces.io.SelectFiles")
190222-14:15:25,513 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.SELECTFILES".
190222-14:15:25,516 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.CREATEDIR" in "/tmp/tmphtg2szek/leads_workflow/_subject_name_LDS0370007..Accelerated_Sagittal_MPRAGE..2018-08-01_10_49_41.0..S711583..LEADS_LDS0370007_MR_Accelerated_Sagittal_MPRAGE__br_raw_20180801153311710_

190222-14:15:47,567 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540980:	NFrames           1
190222-14:15:47,569 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540980:	SliceArraylSize   1
190222-14:15:47,569 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540980:	IsMosaic          0
190222-14:15:47,570 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540980:	ImgPos             98.7542 146.4407 122.8136 
190222-14:15:47,571 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540980:	VolRes              1.0000   1.0000   1.0000 
190222-14:15:47,572 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540980:	VolDim            240      256      208 
190222-14:15:47,573 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540980:	Vc                 -0.0000  -1.0000   0.0000 
190222-14:15:47,574 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540980:	Vr                 -0.0000  -0.0000  -1.0000 
190222-14:15:47,575 nipype.interface INFO:
	 stdout 2019-02-22T14:15:47.540